# Imports

In [1]:
import re
import string

import numpy as np
import pandas as pd
pd.set_option('max_colwidth', 150)

from preprocessing import tokenize

In [2]:
DATA_PATH_PREP = '../DATA/prepared'

# Load data

In [3]:
df_full = pd.read_pickle(f'{DATA_PATH_PREP}/02_df_full_text_no_meta.pkl')
df_full.head()

,author,title,download_link,local_filename,text
0,ivan_vazov,Чичовци,https://chitanka.info/text/3757-chichovtsi.txt.zip,../DATA/original/ivan_vazov_tbc_chichovtsi.txt,\n\tГалерия от типове и нрави български в турско време\n\n\n\tI. Общество\n\n\tУтринното лятно слънце изскокна високо над Стара планина. Потоци от...
1,ivan_vazov,Под игото,https://chitanka.info/text/3753-pod-igoto.txt.zip,../DATA/original/ivan_vazov_tbc_pod_igoto.txt,"\n\n\n\tЧаст първа\n\n\tI. Гост\n\n\tТая прохладна майска вечер чорбаджи Марко, гологлав, по халат, вечеряше с челядта си на двора.\n\tГосподарска..."
2,ivan_vazov,Българският език,https://chitanka.info/text/5189-bylgarskijat-ezik.txt.zip,../DATA/original/ivan_vazov_bylgarskijat_ezik.txt,"\n\n\n\n\tЕзик свещен на моите деди\n\tезик на мъки, стонове вековни,\n\tезик на тая, дето ни роди\n\tза радост не — за ядове отровни.\n\n\tЕзик п..."
3,ivan_vazov,Кочо,https://chitanka.info/text/3851-kocho.txt.zip,../DATA/original/ivan_vazov_kocho.txt,"\n\tЗащитата на Перущица\n\n\n\tO, движенье славно, о, мрачно движенье,\n\tдни на борба горда, о, дни на паденье!\n\tЕпопея тъмна, непозната нам,\..."
4,ivan_vazov,Левски,https://chitanka.info/text/3849-levski.txt.zip,../DATA/original/ivan_vazov_levski.txt,"\n\n\n\tМанастирът тесен за мойта душа е.\n\tКога човек дойде тук да се покае,\n\tтрябва да забрави греховния мир,\n\tда бяга съблазни и да търси ..."


# Create samples

In [4]:
df_group_by_author = df_full.groupby("author")["text"].agg(" ".join).reset_index()
df_group_by_author

,author,text
0,aleko-konstantinov,"\n\n\n\tНашият помощник на регистратора е страшен комик. Изпокапваме от смях, когато почне да бомбардира несправедливата съдба.\n\t— Е, дявол да г..."
1,dimityr-dimov,"\n\n\n\tI глава\n\n\tГроздоберът беше към края си.\n\tОт малките вили и кирпичените постройки сред лозята се разнасяха ту задружни песни, бързи и ..."
2,dimityr-talev,"\n\n\n\tПърва част\n\tХаджи Серафимовата внука\n\n\n\n\tОвде дърво столовито,\n\tстоловито, грановито,\n\tгранки му са до небеси,\n\tа корени — су..."
3,elin-pelin,"\n\n\n\tПърва глава\n\n\tНай-заможният човек в селото беше дядо Йордан Геракът. Пъргав и трудолюбив, той бе работил през целия си живот и бе сполу..."
4,ivan_vazov,\n\tГалерия от типове и нрави български в турско време\n\n\n\tI. Общество\n\n\tУтринното лятно слънце изскокна високо над Стара планина. Потоци от...
5,jordan-jovkov,"\n\n\n\tI\n\n\tГрадът бързо се подновяваше и растеше. Но все пак оставаха много улици, където още можеше да се видят тихи, едновремешни къщурки, с..."


In [5]:
limit_x = df_group_by_author['text'].map(lambda t: len(t.strip().split())).min() - 1000
print(f'{limit_x=}')
limit_y = 150_000

num_authors = df_group_by_author.shape[0]
num_samples = 100

start_idxs = np.random.randint(0, limit_x, (num_authors, num_samples))
sample_lengths = np.random.randint(0, limit_y, (num_authors, num_samples))

df_samples = pd.DataFrame(columns=['text', 'author'])
for i in range(num_authors):
    author_name = df_group_by_author['author'][i]
    for j, start_idx in enumerate(start_idxs[i]):
        sample = df_group_by_author['text'][i][start_idx : (start_idx + sample_lengths[i, j])]
        df_samples.loc[len(df_samples)] = [sample, author_name]
df_samples

limit_x=21623


,text,author
0,"Вярвай Бога, никой път не съм бил опозиция. На, питай когото щеш. И на господина Стамболова съм подавал палтото и сега пак го подавам на нашия мин...",aleko-konstantinov
1,"— щяхме, знайш, да дигнем някоя гюрултийка из Македония, революцийка кое, туй-онуй и може би ще излезе нещо за пред кума. Ама не му е времето, мо...",aleko-konstantinov
2,"се изкикотя над всички тия глупци…\n\t— Налей още една чаша. Наливай, наливай! Още… още…\n\t— Ще прелее.\n\t— Нищо, нека прелей, наливай! Искам д...",aleko-konstantinov
3,"а жандарма, които бяха тласкани да се скитат из разни краища на България, които бяха измъчени, изтезавани, разорени, опропастени. Нека му мислят о...",aleko-konstantinov
4,"та гара. Ний с бай Ганя влязохме в бюфета. Като знаех, че тук ще се бавим цял час, аз преспокойно се разположих при една маса и си поръчах закуска...",aleko-konstantinov
...,...,...
595,"мо тая наука трябваше на Люцкана. И благодарение на възторжената и природна любов, която имаше към професията си и към цветята, той тъй добре усво...",jordan-jovkov
596,"еше застрашително палтото си, наметнато само върху едното рамо, но всичките му буйства се свършваха с много изпотрошени шишета и чаши, или пък няк...",jordan-jovkov
597,"коло изложената навън стока се трупаха многобройни групи селяни. По сички посоки тичаха и се разминаваха хора, тъй унесени в грижите си, че не сва...",jordan-jovkov
598,"разменяха по някоя дума. И сред кипежа и глъчката на тая навалица, сред която звънтяха пари, правеха се шумни пазарлъци, прехвърляха се стоки от е...",jordan-jovkov


# Tokenize

In [6]:
test = 'Това е низ! Съдържа препинателни "знаци" и цифри като: 42, 420, 4200 и 7.\nКак можем да го разделим??'

print('################ Soft ################')
print(tokenize(test, 'soft'))
print()

print('################ Medium ################')
print(tokenize(test, 'medium'))
print()

print('################ Hard ################')
print(tokenize(test, 'hard'))

################ Soft ################
['това', 'е', 'низ', '!', 'съдържа', 'препинателни', '"', 'знаци', '"', 'и', 'цифри', 'като', ':', '42', ',', '420', ',', '4200', 'и', '7', '.', 'как', 'можем', 'да', 'го', 'разделим', '?', '?', 'това е', 'е низ', 'низ !', '! съдържа', 'съдържа препинателни', 'препинателни "', '" знаци', 'знаци "', '" и', 'и цифри', 'цифри като', 'като :', ': 42', '42 ,', ', 420', '420 ,', ', 4200', '4200 и', 'и 7', '7 .', '. как', 'как можем', 'можем да', 'да го', 'го разделим', 'разделим ?', '? ?', 'това е низ', 'е низ !', 'низ ! съдържа', '! съдържа препинателни', 'съдържа препинателни "', 'препинателни " знаци', '" знаци "', 'знаци " и', '" и цифри', 'и цифри като', 'цифри като :', 'като : 42', ': 42 ,', '42 , 420', ', 420 ,', '420 , 4200', ', 4200 и', '4200 и 7', 'и 7 .', '7 . как', '. как можем', 'как можем да', 'можем да го', 'да го разделим', 'го разделим ?', 'разделим ? ?', 'това е низ !', 'е низ ! съдържа', 'низ ! съдържа препинателни', '! съдържа препин

In [8]:
df_full_soft = df_samples.copy()
df_full_soft['text'] = df_full_soft['text'].map(lambda t: tokenize(t, 'soft'))

df_full_med = df_samples.copy()
df_full_med['text'] = df_full_med['text'].map(lambda t: tokenize(t, 'medium'))

df_full_hard = df_samples.copy()
df_full_hard['text'] = df_full_hard['text'].map(lambda t: tokenize(t, 'hard'))

In [9]:
print('################ Heads ################')
print(df_full_soft['text'][0][:20])
print(df_full_med['text'][0][:20])
print(df_full_hard['text'][0][:20])

print('################ Tails ################')
print(df_full_soft['text'][0][-20:])
print(df_full_med['text'][0][-20:])
print(df_full_hard['text'][0][-20:])

################ Heads ################
['вече', 'дърва', '.', 'разгорещих', 'се', '.', ')', 'чувай', 'какво', 'ще', 'ти', 'кажа', 'аз', 'тебе', ':', 'покорна', 'глава', 'сабя', 'я', 'не']
['вече', 'дърво', '.', 'разгорещя', 'се', '.', ')', 'чувам', 'какъв', 'ще', 'ти', 'кажа', 'аз', 'тебе', ':', 'покорен', 'глава', 'сабя', 'я', 'не']
['дърв', 'разгорещ', 'чува', 'кажа', 'тебе', 'покор', 'глава', 'сабя', 'сека', 'разб', 'ний', 'докра', 'пока', 'султа', 'кротъ', 'вяра', 'онз', 'бунтовниц', 'разбира', 'гърчул']
################ Tails ################
['нашия лексикон , забравя', 'лексикон , забравя се', ', забравя се )', 'забравя се ) ,', 'се ) , не', ') , не гледай', ', не гледай ,', 'не гледай , че', 'гледай , че се', ', че се увиват', 'че се увиват около', 'се увиват около тебе', 'увиват около тебе .', 'около тебе . защо', 'тебе . защо се', '. защо се увиват', 'защо се увиват ?', 'се увиват ? мигар', 'увиват ? мигар доброто', '? мигар доброто т']
['нашия лексикон , забравям', 'лексико

In [10]:
df_full_hard.head()

,text,author
0,"[дърв, разгорещ, чува, кажа, тебе, покор, глава, сабя, сека, разб, ний, докра, пока, султа, кротъ, вяра, онз, бунтовниц, разбира, гърчул, река, от...",aleko-konstantinov
1,"[сар, разсил, мачка, фелдфеб, разбира, оцен, па, мисл, зная, какъв, чудо, гоня, мисл, иска, става, начални, подначални, съвсем, зор, изтика, регис...",aleko-konstantinov
2,"[оставя, управит, оценит, митниц, па, ела, хортува, сетн, задръст, евреин, солун, петдесет, шейсет, зная, ама, търгов, хор, доя, оста, наша, софий...",aleko-konstantinov
3,"[пратя, дватр, корпус, хем, няко, пара, капн, хем, македони, свобо, залов, работ, нека, довър, докра, нека, дотъкм, санстефанск, па, видя, благода...",aleko-konstantinov
4,"[найсетн, ний, споко, ний, спасен, веднаж, живе, амнист, нека, мисл, онз, тътр, участъ, гня, подзем, съд, присъд, спя, гол, дъск, око, дененощ, ст...",aleko-konstantinov


# Saving to files

In [ ]:
df_full_soft.to_pickle(f'{DATA_PATH_PREP}/03_df_full_soft.pkl')
df_full_med.to_pickle(f'{DATA_PATH_PREP}/03_df_full_med.pkl')
df_full_hard.to_pickle(f'{DATA_PATH_PREP}/03_df_full_hard.pkl')